In [4]:
# System 
import os
import datetime
import time
import copy
import shutil
# import sys
# sys.path.append('/cnrm/pastel/USERS/lecestres/analyse/')

In [15]:
### EXPERIENCE NAME
#expname = "ocean_reanalysis_GREP"
expname = "sst_retroprevision_sys7"

### PERCENTILE THRESHOLD
percent_thresh = 95
#percent_thresh = 90

### MINIMAL DURATION OF A HW
duration_min = 5
#duration_min = 3

### REGION OF EXPERIENCE

#reg_name = 'north_pacific'
#reg_name = 'north_atlantic'
#reg_name = 'indian_ocean'
#reg_name = 'austral_ocean'
#reg_name = 'tropical_atlantic'
#reg_name = 'tropical_pacific'
#reg_name = 'mediterranee'
reg_name = 'global'

if reg_name == 'north_pacific':
    lats_bnds = np.array([30,65])
    lons_bnds = np.array([120, -120])
if reg_name == 'north_atlantic':
    lats_bnds = np.array([30,65])
    lons_bnds = np.array([-80, 0])
if reg_name == 'indian_ocean':
    lats_bnds = np.array([-30,30])
    lons_bnds = np.array([45, 110])
if reg_name == 'austral_ocean':
    lats_bnds = np.array([-90,-30])
    lons_bnds = np.array([-180, 180])
if reg_name == 'tropical_atlantic':
    lats_bnds = np.array([-30,30])
    lons_bnds = np.array([-70, 20])
if reg_name == 'tropical_pacific':
    lats_bnds = np.array([-30,30])
    lons_bnds = np.array([120, -70])
if reg_name == 'mediterranee':
    lats_bnds = np.array([30,50])
    lons_bnds = np.array([-5, 40])
if reg_name == 'global':
    lats_bnds = np.array([-90,90])
    lons_bnds = np.array([-180,180])

nlat = lats_bnds[1]-lats_bnds[0]
if lons_bnds[1]<lons_bnds[0]:
    nlon = lons_bnds[1]%360-lons_bnds[0]
else:
    nlon = lons_bnds[1]-lons_bnds[0]
#print('nlon : ', nlon)

### YEARS
if expname == 'ocean_reanalysis_GREP':
    end_year=2016
    start_year=1993
elif expname == 'sst_retroprevision_sys7':
    end_year=2016
    start_year=1993
nyear=end_year-start_year+1

### SEASON
#season = "NDJFMAM"
season = "DJF"
if season == 'NDJFMAM':
    nday = 211
    season_start_day = [11,1] #1stNov
    season_end_day = [5,31] #31stMay
    first_day = 0
    
elif season == 'DJF':
    nday = 90
    season_start_day = [12,1] #1stDec
    season_end_day = [3,1] #1stMarch
    first_day = 30
    
ndayseas = nday//duration_min +1

if expname == "ocean_reanalysis_GREP":
    ### NAME OF VARIABLE IN THE NC FILE
    varname = 'thetao_mean'
    
    ### NUMBER OF MEMBS
    first_memb = 0
    last_memb = 1
    nmemb = last_memb-first_memb
    
    ### CROSS VALIDATION
    cv = True
    if cv:
        cv_str = "CV"
    else:
        cv_str = 'notCV'
        
elif expname == "sst_retroprevision_sys7":
    ### NAME OF VARIABLE IN THE NC FILE
    varname = 'sst'
    
    ### NUMBER OF MEMBS
    first_memb = 0
    last_memb = 2
    nmemb = last_memb-first_memb
    
    ### CROSS VALIDATION
    cv = True
    if cv:
        cv_str = "CV"
    else:
        cv_str = 'notCV'

### PARAMETERS
# Will be initialized later in the export
# parameters_str = reg_name + "_" + season + "_" + cv_str + '_percent%i'%(percent_thresh) + '_daymin%i'%(duration_min) +  "ref%i-%i"%(start_year, end_year)

In [11]:
import requests

time_start = time.time()
dataset = 'https://nextcloud.dkrz.de/s/FXoq6ixE54iBfgo/download' # tmax netCDF on DKRZ data repoistory
# dataset = 'http://esgf3.dkrz.de/thredds/fileServer/cmip6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp126/r1i1p1f1/Ofx/areacello/gn/v20190710/areacello_Ofx_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn.nc'
response = requests.get(dataset)

In [12]:
fileGrep = open('testnetCDF.nc', 'wb')
fileGrep.write(response.content)
fileGrep.close

<function BufferedWriter.close>

In [16]:
fileGrep = '/Users/nilshempelmann/birdhouse/testdata/download_2015.nc'
varf = netCDF4.Dataset(fileGrep)
x0,x1 = lons_bnds[0], lons_bnds[1]
cross_greenwich = (x0<0 and x1>0)

In [18]:
if x0<0:
    x0=x0+360
if x1<=0:
    x1=x1+360
y0,y1 = lats_bnds[0], lats_bnds[1]
y_down,y_up = 90-y0, 90-y1
lats_reg = varf.variables['latitude'][y0+90:y1+90]
if cross_greenwich:
    lons_reg_W = varf.variables['longitude'][x0:360]
    lons_reg_E = varf.variables['longitude'][0:x1]
    lons_reg=np.ma.concatenate((lons_reg_W, lons_reg_E))
else:
    lons_reg = varf.variables['longitude'][x0:x1]

In [26]:
varname = 't2m'
target=np.ma.zeros((nyear, nday, nmemb, nlat, nlon))
for j in enumerate(range(first_memb, last_memb)):
    for i,iyear in enumerate(range(start_year, end_year+1)):
        #print(i, iyear)
        i_start_day = (date(iyear,season_start_day[0], season_start_day[1])-date(start_year,1,1)).days
        i_end_day = (date(iyear+1,season_end_day[0], season_end_day[1])-date(start_year,1,1)).days

        if cross_greenwich:
            #print(x0,x1)
            target_W=varf.variables[varname][i_start_day:i_end_day,:,x0:360][:nday,:,:]
            target_E=varf.variables[varname][i_start_day:i_end_day,:,0:x1][:nday,:,:]
            #print(target_W.shape, target_E.shape)
            target_lon=np.flip(np.ma.concatenate((target_W,target_E), axis=2), axis=1)
        else:
            target_lon=np.flip(varf.variables[varname][i_start_day:i_end_day,:,x0:x1][:nday,:,:],axis=1)
        target[i,:,j,:,:]=target_lon[:,y_up:y_down,:]
        print((j*nyear+i+1)/(nyear*nmemb)*100, ' % done after : ', time.time()-time_start)       

ValueError: shape mismatch: value array of shape (31,180,360) could not be broadcast to indexing result of shape (2,90,180,360)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

In [ ]:
if expname == "ocean_reanalysis_GREP":
    fileGrep = "/cnrm/pastel/USERS/lecestres/NO_SAVE/data/ocean_reanalysis_GREP/sst_GREP_1d_199301_201812.nc"
                 
elif expname == "sst_retroprevision_sys7":
    pathPrevi = "/cnrm/pastel/DATA/sys7/series/sst/HK/"
    files = glob(pathPrevi+"*.nc")
    files.sort()
    target = np.ma.zeros((nyear, nday, nmemb, nlat, nlon))
    for i,iyear in enumerate(range(start_year, end_year+1)):
        varf = netCDF4.Dataset(files[i])
        vararray, lats_reg, lons_reg = extract_array(varf, varname, nday,  np.array(lons_bnds), np.array(lats_bnds), start_time = first_day)
        varf.close()
        for j,jmemb in enumerate(range(first_memb, last_memb)):
            target[i,:,j,:,:] = vararray[:,jmemb,:,:]    
            target.mask[i,:,j,:,:] = vararray.mask[:,jmemb,:,:]
            print((i*nmemb+j+1)/(nyear*nmemb)*100, ' % done after : ', time.time()-time_start)

In [3]:
# Heatwave specific
from function_read import *
from HW_detection import *
from HWMI import *

In [ ]:
# Analytic 
import numpy as np
import calendar
import locale
locale.setlocale( locale.LC_ALL , 'en_US' )
from netCDF4 import num2date, date2num 
import netCDF4
from math import sin, cos, sqrt, atan2, radians
from sklearn.neighbors import DistanceMetric
from math import radians

from joblib import Parallel, delayed
import joblib

In [ ]:
# Visualisation
import matplotlib
#matplotlib.use('Agg')
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
# from mpl_toolkits.basemap import Basemap, shiftgrid